<a href="https://colab.research.google.com/github/yueop/AS_LAB/blob/main/models_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive

# 1. 드라이브 마운트
drive.mount('/content/drive')

# 2. 경로 설정
PROJECT_PATH = '/content/drive/MyDrive/AS_LAB'
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)

# 3. 시스템 경로 추가 및 작업 디렉토리 변경
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

print(f"현재 위치: {os.getcwd()}")

Mounted at /content/drive
현재 위치: /content/drive/MyDrive/AS_LAB


In [ ]:
# 파일 이름을 명시적으로 지정
FILE_NAME = "models.py"
# 전체 경로 사용
SAVE_PATH = f"/content/drive/MyDrive/AS_LAB/{FILE_NAME}"

In [ ]:
content = """
import torch
import torch.nn as nn
import torch.nn.functional as F
#모델(MLP, CNN) 구성

#MLP (Linear -> ReLU -> Linear -> ReLU -> Linear)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.0):
        super(MLP, self).__init__()

        self.input_fc = nn.Linear(input_size, hidden_size)
        self.hidden_fc1 = nn.Linear(hidden_size, hidden_size)
        self.output_fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x): #순전파
        x = x.view(x.size(0), -1) #평탄화(MNIST 데이터셋 사용시 4차원 텐서 2차원으로 변환 필요), IRIS 데이터 셋에서 사용 시에는 평탄화가 진행되지 않는다.

        x = F.relu(self.input_fc(x)) #입력층 -> 은닉층1
        x = self.dropout(x)
        x = F.relu(self.hidden_fc1(x)) #은닉층1 -> 은닉층2
        x = self.dropout(x)
        x = self.output_fc(x) #은닉층2 -> 출력층
        return x

#CNN Baseline ((Conv-ReLU-Pool)*2 + FC)
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Linear(32 * 7 * 7, num_classes)

    def forward(self, x): #순전파
        if x.dim() == 2:
            x = x.view(-1, 1, 28, 28)
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) #Flatten
        out = self.fc(out)
        return out

#ImprovedCNN((Conv-BatchNorm-ReLU-Dropout-Pool)*2 + FC)
class ImprovedCNN(CNN):
    def __init__(self, num_classes=10, dropout_rate=0.25):
        super(ImprovedCNN, self).__init__(num_classes) #상속받은 부모 클래스 초기화

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        #FC Layer는 Baseline(부모 클래스) 그대로 사용

#DeepCNN(3-Layer)
class DeepCNN(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.25):
        super(DeepCNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_rate),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_rate),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_rate),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(128 * 3 * 3, num_classes)

    def forward(self, x):
        if x.dim() == 2:
            x = x.view(-1, 1, 28, 28)

        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1) #평탄화
        out = self.fc(out)
        return out


def get_model(model_name, input_size, hidden_size, num_classes, dropout_rate=0.0):
    model_name = model_name.lower()
    if model_name == "mlp":
        return MLP(input_size, hidden_size, num_classes, dropout_rate)
    elif model_name == "cnn":
        return CNN(num_classes=num_classes)
    elif model_name == "improvedcnn":
        return ImprovedCNN(num_classes=num_classes, dropout_rate=dropout_rate)
    elif model_name == "deepcnn":
        return DeepCNN(num_classes=num_classes, dropout_rate=dropout_rate)
    else:
        raise ValueError("존재하지 않는 모델입니다.")
"""

try:
    with open(SAVE_PATH,'w', encoding='utf-8') as f:
        f.write(content.strip())
        print(f"파일이 안전하게 생성되었습니다: {SAVE_PATH}")
except Exception as e:
    print(f"저장 실패: {e}")

파일이 안전하게 생성되었습니다: /content/drive/MyDrive/AS_LAB/models.py
